# Project 3 - Tennis (Collaboration and Competition)



### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`


In [2]:
env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64", no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
            
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


# Assignment Starts

## Approach

We will modify the `DDPG` implementation of the previous P2 assignment in order to satisfy the assumptions of `MADDPG`:
* agents can share information while training their respective policies.
* however at runtime, each agent acts independently, without knowledge of each other.

To that end, we will make the following modifications:
* During training each agent has access to a **shared** replay buffer.
* The **critic** network will take into account states/actions from all agents during training.


### Define Actor and Critic Networks

The `Actor` network is as per origina, DDPG assignment, described here:
* https://github.com/bohana/udacity-deep-rl/blob/master/p2_control/README.md

The `Critic` network contains modifications in order to learn from other agent's experiences. Namely: it is intialized with an additional `n_agents` argument, in order to size the input dimensions accordingly.

In [6]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)


class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        #self.bn = nn.BatchNorm1d(state_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        #state = self.bn(state)
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        
        # the output for each state is a continous function expressed by tanh non linearilty
        return torch.tanh(self.fc3(x))

In [8]:
class Critic(nn.Module):
    """
        Critic (Value) Model.
        
        On the multi-agent scenario (n_agents > 1), the critic also factors in states/actions from the other agents.
    """

    def __init__(self, state_size, action_size, seed, n_agents, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size*n_agents, fc1_units)
        # MADDPG - input to critic also takes into account "other" states and actions
        self.fc2 = nn.Linear(fc1_units + (action_size)*n_agents, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.bn = nn.BatchNorm1d(state_size*n_agents)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, states, actions):
        """Forward pass - uses stats/actions from all agents"""
        if BATCH_NORM:
            state = self.bn(states)
            
        xs = F.relu(self.fcs1(states))
        x = torch.cat((xs, actions), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

### Replay Buffer and OU Noise Process

For multi-agent, the replay buffer stores additional information from the "other" agent. This is reflected in the extended experience tuple defintion:

```python
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward",
                                                                "next_state", "done",
                                                                "other_states", "other_actions", 
                                                                "other_next_states"])
       
```

In [9]:
import numpy as np
import random
import copy
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

In [10]:
class ReplayBuffer:
    """MADDPG-friendly Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        
        # For MADDPG, our experience buffer also keeps track of actions/states from the "other" agent
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done",
                                                                "other_states", "other_actions", "other_next_states"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done, others_state,
            others_action, others_next_state):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done, others_state,
                            others_action, others_next_state)
        
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        other_states = torch.from_numpy(np.vstack([e.other_states for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        other_actions = torch.from_numpy(np.vstack([e.other_actions for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        other_nexts = torch.from_numpy(np.vstack([e.other_next_states for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones, other_states, other_actions, other_nexts)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### OU Noise

The `OUNoise` class implements a random noise process based on Ornstein-Uhlenbeck process. The core idea is to provide randomness correlated with past randomly generated input.

* See also: https://www.quora.com/Why-do-we-use-the-Ornstein-Uhlenbeck-Process-in-the-exploration-of-DDPG


In [11]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = seed
        np.random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
            
        dx = self.theta * (self.mu - x) + self.sigma * np.random.standard_normal(len(x))
        self.state = x + dx
        return self.state

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size, buffer, random_seed, n_agents, fc1_units=64, fc2_units=64):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, random_seed,
                                 fc1_units=fc1_units, fc2_units=fc2_units).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed,
                                  fc1_units=fc1_units, fc2_units=fc2_units).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(),
                                          lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size,
                                   action_size,
                                   random_seed,
                                   n_agents,
                                   fc1_units=fc1_units,
                                   fc2_units=fc2_units).to(device)
        
        self.critic_target = Critic(state_size,
                                    action_size,
                                    random_seed,
                                    n_agents,
                                    fc1_units=fc1_units,
                                    fc2_units=fc2_units).to(device)
        
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(),
                                           lr=LR_CRITIC,
                                           weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, random_seed, theta=NOISE_THETA, sigma=NOISE_SIGMA)
        self.noise_eps = NOISE_EPS_START

        # Replay memory - use shared cache passed as argument
        self.memory = buffer
        
        self.train_counter = 0
    
    def step(self, state, action, reward, next_state, done, others_state,
             others_action, others_next_state, agent_id):
        
        # when saving, make sure agent - "other" mapping is invariant
        if agent_id == 0:
            self.memory.add(state, action, reward, next_state, done,
                            others_state, others_action, others_next_state)
        else:
            self.memory.add(others_state, others_action, reward, others_next_state, done,
                            state, action, next_state)
            
        
        self.train_counter += 1

        # Learn, if enough samples are available in memory
        if len(self.memory) > (UPDATE_EVERY * BATCH_SIZE) and (self.train_counter >= UPDATE_EVERY):
            for _ in range(LEARN_ROUNDS):
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)
            self.train_counter = 0
            
            self.noise_eps -= NOISE_EPS_DECAY
            self.noise_eps = max(NOISE_EPS_MIN, self.noise_eps)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
            
        self.actor_local.train()

        if add_noise:
            action += self.noise_eps * self.noise.sample()
            
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        (states, actions, rewards, next_states, dones, other_states, 
         other_actions, other_next_states) = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get batch predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        
        # concatenate for critic features
        states_all = torch.cat((states, other_states), dim=1).to(device)
        actions_next_all = torch.cat((actions_next, other_actions), dim=1).to(device)
        actions_all = torch.cat((actions, other_actions), dim=1).to(device)
        
        Q_targets_next = self.critic_target(states_all, actions_next_all)
        
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        
        # Compute critic loss
        Q_expected = self.critic_local(states_all, actions_all)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        #other_actions_pred = self.other.actor_local(other_states)
        
        actor_loss = -self.critic_local(states_all,
                                        torch.cat((actions_pred, other_actions), dim=1).to(device)).mean()
        
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        # gradient cliping
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [13]:
class MADDPGAgents:
    def __init__(self, state_size, action_size, n_agents, seed):
        self.n_agents = n_agents
        
        self.shared_buffer = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, SEED)

        self.agents = [Agent(state_size=state_size,
                             action_size=action_size,
                             buffer=self.shared_buffer,
                             random_seed=SEED,
                             n_agents=n_agents,
                             fc1_units=LAYER1_NODES,
                             fc2_units=LAYER2_NODES) for _ in range(n_agents)]
    
    def reset(self):
        for agent in self.agents:
            agent.reset()
    
    def act(self, states):
        actions = [agent.act(state) for agent, state in zip(self.agents, states)]
        return actions
    
    def step(self, states, actions, rewards, next_states, dones):
        '''Process a step using Unity data
           - Unity will send its states/actions/etc... as [n_agents x X] arrays
             in this method, we unpack this array so we can distinguish this vs "other" agents
        '''
        
        for i, e in enumerate(zip(self.agents, states, actions, rewards, next_states,
                                  dones)):
            
            agent, state, action, reward, next_state, done = e
            
            # each state and action comprises all agents, filter which one to use
            other_idx = np.array([o for o in range(self.n_agents) if o != i])[0]
            others_state = states[other_idx]
            others_action = actions[other_idx]
            others_next_state = next_states[other_idx]
            
            # send this/other states/actions for the learning step
            agent.step(state, action, reward, next_state, done, others_state,
                       others_action, others_next_state, agent_id=i)
    
    def save(self):
        for n, ag in enumerate(self.agents):
            torch.save(ag.actor_local.state_dict(), 'checkpoint_ag{}_actor.pth'.format(n))
            torch.save(ag.critic_local.state_dict(), 'checkpoint_ag{}_critic.pth'.format(n))
            

Noting that for scoring, we use the same approach described in the Udacity "benchmark implementation", which states:

```
maximum score over both agents, for each episode, and the orange line shows the average score (after taking the maximum over both agents) over the next 100 episodes.
```

In [14]:
def maddpg_unity(env, mad_agent, n_eps=1000, max_t=1000, print_every=100):
    scores_deque = deque(maxlen=print_every)
    eps_deque = deque(maxlen=print_every)
    
    scores = []
    
    for eps in range(n_eps):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        episode_scores = np.zeros(NUM_AGENTS)
        
        mad_agent.reset()

        for t in range(max_t):     
            actions = mad_agent.act(states)
            
            # execute actions in unity
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            # learn
            mad_agent.step(states, actions, rewards, next_states, dones)
            
            states = next_states
            episode_scores += np.array(rewards)
            
            if np.any(dones):
                break 
        
        score = episode_scores.max()
        scores_deque.append(score)
        scores.append(score)
        eps_deque.append(t + 1)
        
        print('\rEpisode {}\tAverage of Max Scores: {:.3f}'.format(eps,
                                                                   np.mean(scores_deque)), end="")
        
        if eps % print_every == 0:
            print('\rEpisode {}\tAverage of Max Scores: {:.3f}'.format(eps, np.mean(scores_deque)), end="")
            print('; Mean eps len: {:.2f}; Sum scores: {:.2f}; Noise eps: {:.3f}'.format(np.mean(eps_deque),
                                                                                         np.sum(scores_deque),
                                                                                         mad_agents.agents[0].noise_eps))
            #print('Last actions:', actions)
            
        if np.mean(scores_deque) >= STOP_AT:
            break
    
    mad_agents.save()
    return scores

### Hyper Parameters

Using noise decay suggestion from:
https://towardsdatascience.com/training-two-agents-to-play-tennis-8285ebfaec5f

In [15]:
BUFFER_SIZE = 150000    # replay buffer size
BATCH_SIZE = 100        # minibatch size
GAMMA = 0.99           # discount factor
TAU = 0.001            # for soft update of target parameters
LR_ACTOR = 0.001      # learning rate of the actor 
LR_CRITIC = 0.001     # learning rate of the critic
WEIGHT_DECAY = 0       # L2 weight decay

N_EPS = 5000
MAX_T = 1000

UPDATE_EVERY = 15       # num of timesteps to run learning
LEARN_ROUNDS = 10       # repeat learning (sample from memory + backprop) this many times 

LAYER1_NODES=300       # number of units in hidden layer 1 (both networks)
LAYER2_NODES=100       # number of units in hidden layer 2 (both networks)

#
# noise parameters for OU process
#
NOISE_THETA=0.45
NOISE_SIGMA=0.55

# noise decay
NOISE_EPS_START=1.0
NOISE_EPS_MIN=1.0
NOISE_EPS_DECAY=0.0


BATCH_NORM = True
STOP_AT = 0.5

NUM_AGENTS = 2


SEED = 13

In [16]:
import matplotlib.pyplot as plt
%matplotlib inline
import time

print('Your device:', device)
print('- training starts', time.asctime())

mad_agents = MADDPGAgents(state_size, action_size, NUM_AGENTS, SEED)
scores = []

# initially, switch off noise decay and populate the replay buffer - this step does not do any learning
NOISE_EPS_DECAY=0.0
UPDATE_EVERY=500
scores += maddpg_unity(env, mad_agents,
                       n_eps=500,
                       max_t=MAX_T,
                       print_every=100)


# reset to normal behavior: learn and decay ou noise
UPDATE_EVERY=15

scores += maddpg_unity(env, mad_agents,
                       n_eps=N_EPS,
                       max_t=MAX_T,
                       print_every=100)

print('\n\n- training end', time.asctime())

Your device: cuda:0
- training starts Mon Mar 18 00:21:24 2019
Episode 0	Average of Max Scores: 0.000; Mean eps len: 14.00; Sum scores: 0.00; Noise eps: 1.000
Episode 100	Average of Max Scores: 0.016; Mean eps len: 17.15; Sum scores: 1.63; Noise eps: 1.000
Episode 200	Average of Max Scores: 0.013; Mean eps len: 17.80; Sum scores: 1.28; Noise eps: 1.000
Episode 300	Average of Max Scores: 0.012; Mean eps len: 17.46; Sum scores: 1.16; Noise eps: 1.000
Episode 400	Average of Max Scores: 0.011; Mean eps len: 17.88; Sum scores: 1.08; Noise eps: 1.000
Episode 0	Average of Max Scores: 0.000; Mean eps len: 14.00; Sum scores: 0.00; Noise eps: 1.000
Episode 100	Average of Max Scores: 0.000; Mean eps len: 14.22; Sum scores: 0.00; Noise eps: 1.000
Episode 200	Average of Max Scores: 0.001; Mean eps len: 14.37; Sum scores: 0.09; Noise eps: 1.000
Episode 300	Average of Max Scores: 0.000; Mean eps len: 14.20; Sum scores: 0.00; Noise eps: 1.000
Episode 400	Average of Max Scores: 0.000; Mean eps len: 14.

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.figure.set_size_inches(12, 5)

plt.plot(np.arange(1, len(scores)+1),
         scores)

plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
import pandas as pd


s = pd.Series(scores)
r = s.rolling(100)

r.mean().dropna().plot(figsize=(12, 5), title='Rolling avg 100eps')

Learning notes

* no improvements: learning stuck due to low exploration -> tweak noise parameters
* training colapse: too much exploration?
